In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os
import h5py
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
import time
from pathlib import Path
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
import warnings
warnings.filterwarnings('ignore')


# -------------------------
#Train ve Test klasörleri
# -------------------------
train_dir = "/kaggle/input/cards-image-datasetclassification/train"
test_dir  = "/kaggle/input/cards-image-datasetclassification/test"

# Veri setini yükleme
batch_size = 32
img_height = 128
img_width  = 128

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Sınıf isimleri
class_names = train_ds.class_names
print("Sınıf isimleri:", class_names)
print("Toplam sınıf sayısı:", len(class_names))

# İlk 24 resmi görselleştirme
plt.figure(figsize=(12, 8))

images_shown = 32
rows = 4
cols = 8

for images, labels in train_ds.take(1):
    for i in range(images_shown):
        plt.subplot(rows, cols, i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.xlabel(class_names[labels[i]])

plt.tight_layout()
plt.show()


# Eğitim ve test etiketlerini listeye çevirme
train_labels = []
for _, labels in train_ds:
    train_labels.extend(labels.numpy())

test_labels = []
for _, labels in test_ds:
    test_labels.extend(labels.numpy())


# Eğitim ve test verisi sınıf dağılımını alt alta görselleştirme
plt.figure(figsize=(10, 10))

plt.subplot(2, 1, 1)
sns.countplot(x=train_labels)
plt.title('Eğitim Verisi Sınıf Dağılımı')
plt.xlabel('Sınıf')
plt.ylabel('Görüntü Sayısı')

plt.subplot(2, 1, 2)
sns.countplot(x=test_labels)
plt.title('Test Verisi Sınıf Dağılımı')
plt.xlabel('Sınıf')
plt.ylabel('Görüntü Sayısı')

plt.tight_layout()
plt.show()


In [ ]:
# -------------------------
# Veri Ön İşleme
# -------------------------

# Eğitim ve test veri setini NumPy dizilerine dönüştürme
X_train_list = []
y_train_list = []

for x, y in train_ds:
    X_train_list.append(x.numpy())
    y_train_list.append(y.numpy())

X_test_list = []
y_test_list = []

for x, y in test_ds:
    X_test_list.append(x.numpy())
    y_test_list.append(y.numpy())

# Batchleri birleştirme
X_train = np.vstack(X_train_list)
y_train = np.hstack(y_train_list)

X_test = np.vstack(X_test_list)
y_test = np.hstack(y_test_list)

# Normalize etme (0-1 aralığı)
X_train = X_train.astype("float32") / 255.0
X_test  = X_test.astype("float32") / 255.0

# One-hot encoding
num_classes = len(class_names)
y_train_categorical = keras.utils.to_categorical(y_train, num_classes)
y_test_categorical  = keras.utils.to_categorical(y_test, num_classes)


# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)

datagen.fit(X_train)


# Kontrol çıktıları
print("Normalizasyon ve one-hot işlemi tamamlandı.\n")
print("Normalize edilmiş eğitim verisi şekli:", X_train.shape)
print("One-hot encoded eğitim etiketleri şekli:", y_train_categorical.shape)
print("Normalize edilmiş test verisi şekli:", X_test.shape)
print("One-hot encoded test etiketleri şekli:", y_test_categorical.shape)

In [ ]:
# -------------------------
# Temel CNN Modeli Oluştur
# -------------------------

# Modeli sıfırlama
cnn_model = models.Sequential()

# İlk evrişim bloğu
cnn_model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Dropout(0.2))

# İkinci evrişim bloğu
cnn_model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Dropout(0.3))

# Üçüncü evrişim bloğu
cnn_model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Dropout(0.4))

# Sınıflandırıcı katmanlar
cnn_model.add(layers.GlobalAveragePooling2D())
cnn_model.add(layers.Dense(128, activation='relu'))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Dropout(0.5))
cnn_model.add(layers.Dense(53, activation='softmax'))  # 53 sınıf

# Model özetini yazdır
cnn_model.summary()

In [ ]:
# -------------------------
# Modeli Derle ve Eğit
# -------------------------

# Modeli derleme
cnn_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# Callbackleri tanımlama
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.0001
)

# Modeli eğitme
start_time = time.time()

history = cnn_model.fit(
    datagen.flow(X_train, y_train_categorical, batch_size=64),
    epochs=50,
    validation_data=(X_test, y_test_categorical),
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

training_time = time.time() - start_time
print(f"\n Eğitim süresi: {training_time:.2f} saniye")

# Eğitim ve Doğrulama Sonuçlarının Görselleştirilmesi
plt.figure(figsize=(12, 4))

# Doğruluk grafiği
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.title('Model Doğruluğu')
plt.xlabel('Epoch')
plt.ylabel('Doğruluk')
plt.legend()
plt.grid(True)

# Kayıp grafiği
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.title('Model Kaybı')
plt.xlabel('Epoch')
plt.ylabel('Kayıp')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# -------------------------
# Model Değerlendirme
# -------------------------

# Test seti üzerinde modeli değerlendirme
test_loss, test_accuracy = cnn_model.evaluate(X_test, y_test_categorical, verbose=0)
print(f"Test Kaybı: {test_loss:.4f}")
print(f"Test Doğruluğu: {test_accuracy:.4f}")


# Test seti tahminleri
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test_categorical, axis=1)


# Sınıflandırma raporu
print("\nSınıflandırma Raporu:")
print(classification_report(y_true, y_pred_classes, target_names=class_names))


# Karmaşıklık matrisi
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Karmaşıklık Matrisi')
plt.ylabel('Gerçek Etiket')
plt.xlabel('Tahmin Edilen Etiket')
plt.show()


# Doğru ve yanlış tahmin örneklerini ayırma
correct_indices = np.where(y_pred_classes == y_true)[0]
incorrect_indices = np.where(y_pred_classes != y_true)[0]

plt.figure(figsize=(12, 8))

# Doğru tahmin örnekleri (ilk 4)
for i, correct in enumerate(correct_indices[:4]):
    plt.subplot(2, 4, i+1)
    plt.imshow(X_test[correct])
    plt.title(f"Tahmin: {class_names[y_pred_classes[correct]]}\nGerçek: {class_names[y_true[correct]]}")
    plt.axis('off')

# Yanlış tahmin örnekleri (ilk 4)
for i, incorrect in enumerate(incorrect_indices[:4]):
    plt.subplot(2, 4, i+5)
    plt.imshow(X_test[incorrect])
    plt.title(f"Tahmin: {class_names[y_pred_classes[incorrect]]}\nGerçek: {class_names[y_true[incorrect]]}")
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Transfer Learning ile VGG16 Modeli

# Önceden eğitilmiş VGG16 modelini kullanma
base_model = keras.applications.VGG16(
    weights='imagenet',       # ImageNet ağırlıkları
    include_top=False,        # Üst sınıflandırıcı katmanları dahil etme
    input_shape=(128, 128, 3) # Card veri setimiz 128x128 boyutunda
)

# Temel modelin katmanlarını donduralım (eğitimlerini engelleme)
for layer in base_model.layers:
    layer.trainable = False

# Yeni transfer learning modeli
transfer_model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')  # 53 sınıf
])

# Modeli derleme
transfer_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

transfer_model.summary()

# Transfer learning modelini eğitme
start_time = time.time()

history_transfer = transfer_model.fit(
    datagen.flow(X_train, y_train_categorical, batch_size=64),
    epochs=20,
    validation_data=(X_test, y_test_categorical),
    verbose=1
)

training_time = time.time() - start_time
print(f"Eğitim süresi: {training_time:.2f} saniye")

# Transfer learning modelini test seti üzerinde değerlendirme
test_loss_transfer, test_accuracy_transfer = transfer_model.evaluate(X_test, y_test_categorical, verbose=0)
print(f"Transfer Learning Test Doğruluğu: {test_accuracy_transfer:.4f}")

In [ ]:
# -------------------------
# Model Karşılaştırması ve Öznitelik Haritaları
# -------------------------

# Model performanslarını karşılaştırma
models = ['Temel CNN', 'Transfer Learning']
accuracies = [test_accuracy, test_accuracy_transfer]

plt.figure(figsize=(8, 6))
bars = plt.bar(models, accuracies, color=['blue', 'orange'])
plt.title('Model Performans Karşılaştırması')
plt.ylabel('Doğruluk')
plt.ylim(0, 1)

# Çubukların üzerine doğruluk değerlerini yazma
for bar, accuracy in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{accuracy:.4f}', ha='center', va='bottom')

plt.show()


# Öznitelik haritalarını görselleştirme
try:
    # Dummy giriş ile modeli çağır (cnn_model.input tanımlamak için)
    dummy_input = tf.random.uniform((1, 128, 128, 3)) 
    _ = cnn_model(dummy_input)

    # İlk 8 katmanın çıktılarını alma
    conv_layers = [layer for layer in cnn_model.layers[:8] if 'conv' in layer.name]
    layer_outputs = [layer.output for layer in conv_layers]

    # Aktivasyon modelini oluşturma
    activation_model = keras.models.Model(inputs=cnn_model.input, outputs=layer_outputs)

    # Test setinden bir görüntü seçme
    img = X_test[10:11]
    activations = activation_model.predict(img)

    # İlk 16 filtreyi görselleştirme
    for layer_name, layer_activation in zip([l.name for l in conv_layers], activations):
        n_filters = layer_activation.shape[-1]
        n_cols = 8
        n_rows = n_filters // n_cols + (n_filters % n_cols > 0)

        plt.figure(figsize=(n_cols*2, n_rows*2))
        plt.suptitle(layer_name)

        for i in range(min(n_filters, 16)):
            plt.subplot(n_rows, n_cols, i+1)
            plt.imshow(layer_activation[0, :, :, i], cmap='viridis')
            plt.axis('off')

        plt.tight_layout()
        plt.show()

except Exception as e:
    print("Öznitelik haritalarını oluştururken hata oluştu:")
    print(e)

In [ ]:
# -------------------------
# Model İyileştirme Teknikleri
# -------------------------

import matplotlib.pyplot as plt
from tensorflow import keras

# Farklı optimizasyon tekniklerini deneme
optimizers = {
    'Adam': keras.optimizers.Adam(learning_rate=0.001),
    'RMSprop': keras.optimizers.RMSprop(learning_rate=0.001),
    'SGD': keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
}

results = {}

for opt_name, optimizer in optimizers.items():
    print(f"\n{opt_name} optimizer ile eğitim...")

    # Yeni bir temel CNN modeli oluştur
    model = keras.Sequential([
        # İlk evrişim bloğu
        keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Dropout(0.2),

        # İkinci evrişim bloğu
        keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Dropout(0.3),

        # Üçüncü evrişim bloğu
        keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Dropout(0.4),

        # Sınıflandırıcı
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(53, activation='softmax')  # bizim veri seti 53 sınıf
    ])

    # Modeli derleme
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Modeli eğitme (10 epoch hızlı test için)
    history = model.fit(X_train, y_train_categorical,
                       epochs=10,
                       validation_data=(X_test, y_test_categorical),
                       verbose=0)

    # Test seti üzerinde doğruluk
    test_loss, test_acc = model.evaluate(X_test, y_test_categorical, verbose=0)
    results[opt_name] = test_acc
    print(f"{opt_name} Test Doğruluğu: {test_acc:.4f}")

# Optimizasyon tekniklerini karşılaştıralım
plt.figure(figsize=(8, 6))
bars = plt.bar(results.keys(), results.values(), color=['blue', 'green', 'orange'])
plt.title('Optimizasyon Teknikleri Karşılaştırması')
plt.ylabel('Doğruluk')
plt.ylim(0, 1)

for bar, accuracy in zip(bars, results.values()):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{accuracy:.4f}', ha='center', va='bottom')

plt.show()

In [ ]:
# -------------------------
# Proje Sonuçlarının Özetlenmesi
# -------------------------

print("=" * 50)
print("İSKAMBİL KARTLARI VERİ SETİ PROJE SONUÇLARI")
print("=" * 50)

# Temel CNN modeli doğruluğu
print(f"Temel CNN Model Doğruluğu: {test_accuracy:.4f}")

# Transfer Learning doğruluğu
print(f"Transfer Learning Model Doğruluğu: {test_accuracy_transfer:.4f}")

# En iyi optimizer ve doğruluk
best_optimizer = max(results, key=results.get)
best_accuracy = results[best_optimizer]
print(f"En iyi optimizer: {best_optimizer} ({best_accuracy:.4f})")

print("=" * 50)